In [1]:
import numpy as np 
import pandas as pd
import os
import html
from tabulate import tabulate
from scipy.sparse import csr_matrix

In [2]:
#El encoding="utf-8" lo que hace es controlar los caracteres raros estilo ñ
anime_cols = ['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members']
ratings_cols = ['user_id', 'anime_id', 'rating']

base_path = os.getcwd()

# Archivos dentro de la carpeta "RecomendacionesAnime" relativa al notebook
anime_file = os.path.join(base_path, "anime.csv")
ratings_file = os.path.join(base_path, "rating.csv")

# Cargar los CSVs
anime = pd.read_csv(anime_file, names=anime_cols, header=0, encoding="utf-8")
ratings = pd.read_csv(ratings_file, names=ratings_cols, header=0, encoding="utf-8")

In [3]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
#El html lo que hace es por si haya algun caracter raro pues lo ponga de forma normal
anime['name'] = anime['name'].apply(html.unescape)

In [5]:
ratings.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


## Limpieza de datos y cambio de nombres

In [6]:
#Se sabe que alguno de estos no los usamos pero los limpiamos y arregalos por si acaso se desea utilizar a futuro
anime['episodes'] = anime['episodes'].replace('Unknown', np.nan).astype(float)
anime['genre'] =  anime['genre'].fillna('Unknown')
anime['rating'] = anime['rating'].fillna(anime['rating'].mean())
anime['type'] = anime['type'].fillna('Unknown')

#Eliminamos y/o tratamos los que tengan alguna columna vacia o en otras diferente (ejemplo un caso de un anime que parte de sus datos estaba en la B del csv)
anime = anime.dropna(subset=['episodes', 'rating', 'type', 'genre'])
ratings = ratings[ratings['rating'] != -1]
ratings = ratings.dropna(subset=['user_id', 'anime_id', 'rating'])

In [7]:
#Cambio de nombres para tenerlos mas claros
ratings = ratings.rename(columns={"rating": "user_rating"})
anime = anime.rename(columns={"rating": "anime_rating"})

In [8]:
ratings_with_name = ratings.merge(anime[['anime_id', 'name']], on='anime_id', how='inner')
#Estas lineas cambian el tipo de datos de las columnas introducidas a category
#Los category ocupan menos espacio que object o int
ratings_with_name['name'] = ratings_with_name['name'].astype('category')
ratings_with_name['user_id'] = ratings_with_name['user_id'].astype('category')
ratings_with_name.head()

,user_id,anime_id,user_rating,name
0,1,8074,10,Highschool of the Dead
1,1,11617,10,High School DxD
2,1,11757,10,Sword Art Online
3,1,15451,10,High School DxD New
4,2,11771,10,Kuroko no Basket


In [9]:
# Filtrar por los animes más puntuados
anime_counts = ratings_with_name['name'].value_counts() #Cuenta de calificación por anime
popular_animes = anime_counts[anime_counts > 400].index #Filtrado de mas de 400 Calificaciones

filtered = ratings_with_name[ratings_with_name['name'].isin(popular_animes)] #Filtrado para solamente quedarse los populares y eliminar el resto

ratings_pivot = filtered.pivot_table(
    index='user_id',
    columns='name',
    values='user_rating',
    aggfunc='mean',
    observed=True
).astype('float32')

ratings_pivot.head()

name,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi","""Bungaku Shoujo"" Memoire","""Bungaku Shoujo"" Movie",.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,.hack//Roots,...,ef: A Tale of Melodies.,ef: A Tale of Melodies. - Prologue,ef: A Tale of Memories.,ef: A Tale of Memories. - Prologue,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
corrMatrix = ratings_pivot.corr(method='pearson', min_periods=250) #Minimo 250 que evaluaron los animes
corrMatrix.head()

In [ ]:
#Usuario de prueba 
user_test_ratings = {
    "Hunter x Hunter (2011)": 10,
    "School Days": 1
}

#Verificar que esos animes existen en la matriz de correlación
available_animes = [anime for anime in user_test_ratings.keys() if anime in corrMatrix.columns]
if not available_animes:
    raise ValueError("Ninguno de los animes especificados está en la matriz de correlación.")

#Convertir a serie con las calificaciones del usuario de prueba
myRatings = pd.Series({anime: user_test_ratings[anime] for anime in available_animes})
print("Calificaciones del usuario de prueba:")
print(myRatings)

#Generar candidatos de recomendación
simCandidates = pd.Series(dtype='float64')

for anime_name, rating_value in myRatings.items():
    print(f"\nAñadiendo animes similares a {anime_name}...")
    #Recuperar las similitudes del anime actual
    sims = corrMatrix[anime_name].dropna()
    #Escalar la similaridad multiplicando la correlación por la calificación de la persona
    sims = sims.map(lambda x: x * rating_value)
    #Agregar al conjunto de candidatos
    simCandidates = pd.concat([simCandidates, sims])

#Agrupar y ordenar resultados
print("\nOrdenando recomendaciones...")
simCandidates = simCandidates.groupby(simCandidates.index).sum() #Agrupamos para sumar puntaje por que salen repetidos
simCandidates.sort_values(inplace=True, ascending=False)

#Eliminar los animes ya vistos/repetidos
filteredSims = simCandidates.drop(myRatings.index, errors='ignore')

print("\nTop 50 recomendaciones para el usuario de prueba:")
print(filteredSims.head(50))